In [4]:
import warnings
warnings.filterwarnings('ignore')
import pyodbc
import pandas as pd
import numpy as np
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Continuous, Categorical, Integer
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import datetime
import plotly.express as px
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import xgboost as xgb


def connect_sqlserver(server='deti-sql-aulas.ua.pt',database='sisqualFORECASTDATA',username='sisqual',password='rh4.0'):
    con = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = con.cursor()

    return con,cursor
con,cursor=connect_sqlserver()

tiendas_query=cursor.execute('SELECT Distinct tienda from Martim.ForecastData;')
tiendas=[item[0] for item in tiendas_query]

print(len(tiendas))
lags=5

121


In [5]:
query='SELECT*FROM [sisqualFORECASTDATA].[Martim].[ForecastData] Where tienda=?;'
data=pd.read_sql(query,con,params=[37])
data['fechaHoraInicio']=pd.to_datetime(data['fechaHoraInicio'])
data['fechaHoraFim']=pd.to_datetime(data['fechaHoraFim'])
data['fecha']=pd.to_datetime(data['fecha'])
data['cantidad']=data['cantidad'].astype(float)
data['numero']=data['numero'].astype(float)
data['tickets']=data['tickets'].astype(float)
data['importe']=data['importe'].astype(float)
data['caja']=data['caja'].astype(int)
data['tickets']=data['tickets'].apply(lambda x:max(x,0))
data

,tienda,caja,fecha,fechaHoraInicio,fechaHoraFim,cantidad,numero,tickets,importe
0,37,7,2020-05-09,2020-05-09 19:00:00,2020-05-09 19:15:00,13.000,9.0,8.0,6.98
1,37,7,2020-05-09,2020-05-09 19:15:00,2020-05-09 19:30:00,14.000,11.0,7.0,8.25
2,37,7,2020-05-09,2020-05-09 19:30:00,2020-05-09 19:45:00,9.000,6.0,1.0,6.05
3,37,7,2020-05-09,2020-05-09 19:45:00,2020-05-09 20:00:00,22.000,16.0,7.0,19.10
4,37,7,2020-05-09,2020-05-09 20:00:00,2020-05-09 20:15:00,6.000,4.0,2.0,3.66
...,...,...,...,...,...,...,...,...,...
421402,37,7,2018-10-02,2018-10-02 17:15:00,2018-10-02 17:30:00,7.000,7.0,6.0,6.15
421403,37,7,2018-10-02,2018-10-02 17:30:00,2018-10-02 17:45:00,10.000,7.0,6.0,7.13
421404,37,7,2018-10-02,2018-10-02 17:45:00,2018-10-02 18:00:00,14.000,10.0,7.0,6.93
421405,37,7,2018-10-02,2018-10-02 18:00:00,2018-10-02 18:15:00,21.000,13.0,10.0,12.31


In [3]:
data=data.drop(columns=['caja'])
forecast_data=pd.DataFrame(data.resample('H', on='fechaHoraInicio').tickets.sum())
forecast_data.reset_index(inplace=True)
forecast_data

,fechaHoraInicio,tickets
0,2015-01-02 10:00:00,199.0
1,2015-01-02 11:00:00,268.0
2,2015-01-02 12:00:00,337.0
3,2015-01-02 13:00:00,390.0
4,2015-01-02 14:00:00,249.0
...,...,...
50784,2020-10-18 10:00:00,106.0
50785,2020-10-18 11:00:00,148.0
50786,2020-10-18 12:00:00,177.0
50787,2020-10-18 13:00:00,151.0


In [5]:
def fechado(tickets):
    ret=None
    
    if tickets==0:
        ret=1
    else:
        ret=0
    return ret

In [7]:
forecast_data['Mês']=forecast_data['fechaHoraInicio'].apply(lambda x: x.month)
forecast_data['Dia']=forecast_data['fechaHoraInicio'].apply(lambda x: x.weekday())
#forecast_data['Dia']=forecast_data['fechaHoraInicio'].apply(lambda x: x.strftime('%A'))
#forecast_data['Mês']=forecast_data['fechaHoraInicio'].apply(lambda x: x.strftime('%B'))
forecast_data['Hora']=forecast_data['fechaHoraInicio'].apply(lambda x: x.hour)
forecast_data['Fechado']=forecast_data['tickets'].apply(lambda x: fechado(x))

inicio_2020=datetime.datetime(2020, 1, 1)
forecast_data=forecast_data[forecast_data['fechaHoraInicio']<inicio_2020]
#forecast_data=pd.get_dummies(forecast_data,prefix=['Fechado'])
forecast_data

,fechaHoraInicio,tickets,Mês,Dia,Hora,Fechado
0,2015-01-02 10:00:00,199.0,1,4,10,0
1,2015-01-02 11:00:00,268.0,1,4,11,0
2,2015-01-02 12:00:00,337.0,1,4,12,0
3,2015-01-02 13:00:00,390.0,1,4,13,0
4,2015-01-02 14:00:00,249.0,1,4,14,0
...,...,...,...,...,...,...
43785,2019-12-31 19:00:00,146.0,12,1,19,0
43786,2019-12-31 20:00:00,0.0,12,1,20,1
43787,2019-12-31 21:00:00,0.0,12,1,21,1
43788,2019-12-31 22:00:00,0.0,12,1,22,1


In [8]:
tickets=np.array(forecast_data['tickets'])

def to_supervised(a,n_lags,n_output=1):
    X,y=[],[]
    
    for i in range(len(a)-n_lags):
        aux=[]
        for j in range(i,i+n_lags,1):
            aux.append(a[j])
        if i+n_lags+n_output<len(a):
            X.append(aux)
            y.append(a[i+n_lags:i+n_lags+n_output])

    return np.array(X),np.array(y)

X,y=to_supervised(tickets,lags)
    

In [9]:
df=pd.DataFrame(np.hstack((X,y)),columns=['lag'+str(i-1) for i in range(lags+1,0,-1)])
df=df.rename(columns={'lag0':'target'})
df

,lag5,lag4,lag3,lag2,lag1,target
0,199.0,268.0,337.0,390.0,249.0,109.0
1,268.0,337.0,390.0,249.0,109.0,86.0
2,337.0,390.0,249.0,109.0,86.0,104.0
3,390.0,249.0,109.0,86.0,104.0,142.0
4,249.0,109.0,86.0,104.0,142.0,171.0
...,...,...,...,...,...,...
43779,435.0,311.0,162.0,157.0,152.0,226.0
43780,311.0,162.0,157.0,152.0,226.0,146.0
43781,162.0,157.0,152.0,226.0,146.0,0.0
43782,157.0,152.0,226.0,146.0,0.0,0.0


In [10]:
forecast_data_final=pd.concat([forecast_data.iloc[lags:,:].reset_index(drop=True),df],axis=1,join="inner")
forecast_data_final=forecast_data_final.drop(columns=['fechaHoraInicio','tickets'])
#forecast_data_final.drop(['lag'+str(i) for i in range(lags-1,0,-1) if i%24!=0], axis=1, inplace=True)
#forecast_data_final=forecast_data_final[['Mês','Dia','Hora','lag168','lag144','lag120','lag96','lag72','lag48','lag24','lag3','lag2','lag1','target']]
forecast_data_final

,Mês,Dia,Hora,Fechado,lag5,lag4,lag3,lag2,lag1,target
0,1,4,15,0,199.0,268.0,337.0,390.0,249.0,109.0
1,1,4,16,0,268.0,337.0,390.0,249.0,109.0,86.0
2,1,4,17,0,337.0,390.0,249.0,109.0,86.0,104.0
3,1,4,18,0,390.0,249.0,109.0,86.0,104.0,142.0
4,1,4,19,0,249.0,109.0,86.0,104.0,142.0,171.0
...,...,...,...,...,...,...,...,...,...,...
43779,12,1,18,0,435.0,311.0,162.0,157.0,152.0,226.0
43780,12,1,19,0,311.0,162.0,157.0,152.0,226.0,146.0
43781,12,1,20,1,162.0,157.0,152.0,226.0,146.0,0.0
43782,12,1,21,1,157.0,152.0,226.0,146.0,0.0,0.0


In [21]:
X=forecast_data_final.iloc[:,:-1].values
y=forecast_data_final.iloc[:,-1].values
X_train=X[:-1440,:]
y_train=y[:-1440]
X_test=X[-1440:]
y_test=y[-1440:]

In [162]:
def feature_selection(X_train,y_train,n_features_to_select,step):
    n_features=X_train.shape[1]
    r_features=list(range(X_train.shape[1]))
    
    
    for i in range(n_features,n_features_to_select,-step):
        #train decision tree
        dtc=DecisionTreeRegressor(random_state=1)
        dtc.fit(X_train,y_train)
        feature_importances_values=dtc.feature_importances_
        feature_importances_keys=[i for i in range(len(feature_importances_values))]
        feature_importances_dic={}
        for i in range(len(feature_importances_keys)):
            feature_importances_dic[feature_importances_keys[i]]=feature_importances_values[i]
            
        #sort
        worse_features=sorted(feature_importances_dic, key=feature_importances_dic.get, reverse=True)[-step:]
        for index in worse_features:
            del r_features[index]
        

        
    return r_features
    

In [13]:
dtr=DecisionTreeRegressor(random_state=1)
selector = RFE(dtr, n_features_to_select=10, step=5)
selector = selector.fit(X_train, y_train)

In [14]:
best_features=np.where(selector.support_)[0]

In [72]:
best_features=list(best_features)

if 0 not in best_features:
    best_features.append(0)
    
if 1 not in best_features:
    best_features.append(1)
best_features=np.array(best_features)

In [73]:
best_features_name=forecast_data_final.columns[best_features]
best_features_name

Index(['Hora', 'Fechado', 'lag168', 'lag167', 'lag147', 'lag144', 'lag27',
       'lag24', 'lag17', 'lag1', 'Mês', 'Dia'],
      dtype='object')

In [74]:
non_lag_features_index=[]
for i in range(len(best_features_name)):
    nome=best_features_name[i]
    if 'lag' not in nome:
        non_lag_features_index.append(i)

In [75]:
lags_=len(best_features)-len(non_lag_features_index)

In [76]:
non_lag_features_index
best_features[non_lag_features_index]

array([2, 3, 0, 1], dtype=int64)

In [79]:
X_train[0].shape

(172,)

In [81]:
best_features_lags=[]
for i in range(len(best_features)):
    if best_features[i] not in best_features[non_lag_features_index]:
        best_features_lags.append(best_features[i])
best_features_lags

[4, 5, 25, 28, 145, 148, 155, 171]

In [86]:
len(best_features)

12

In [157]:
forecast_data_final['Dia']=forecast_data_final['Dia'].astype('category')
forecast_data_final['Mês']=forecast_data_final['Mês'].astype('category')
forecast_data_final['Fechado']=forecast_data_final['Fechado'].astype('category')
forecast_data_final.dtypes

Dia        category
Mês        category
Hora          int64
Fechado    category
lag5        float64
lag4        float64
lag3        float64
lag2        float64
lag1        float64
target      float64
dtype: object

In [46]:
0.999**40000

4.1641756403861e-18

In [118]:
rfr.feature_importances_

array([0.00377864, 0.00784453, 0.13845373, 0.45162681, 0.02805013,
       0.00454116, 0.01971719, 0.00852158, 0.33746622])

In [ ]:
alpha**1000

In [22]:
sample_weight=[]
alpha=0.999
n=X_train.shape[0]

for i in range(n):
    valor=alpha**i
    
    if valor<0.1:
        valor=0.1
    sample_weight.append(valor)
    #sample_weight.append((i+1)/n)


In [23]:
sample_weight=list(reversed(sample_weight))

In [123]:
    max_tienda=cursor.execute('SELECT max(tienda) from Martim.models;')

Error: ('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionWrite (send()). (10054) (SQLExecDirectW); [01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]Erro geral de rede. Consulte a documentação da rede. (11)')

In [ ]:
xbg_ga.

In [33]:
X=forecast_data_final.iloc[:,:-1].values
y=forecast_data_final.iloc[:,-1].values
X_train=X[:-1440,:]
y_train=y[:-1440]
X_test=X[-1440:]
y_test=y[-1440:]

rfr=RandomForestRegressor(bootstrap=True ,max_features=0.8, min_samples_leaf=1, min_samples_split=13, n_estimators=100,oob_score=True)
param_grid={'learning_rate':Continuous(0.001,0.9),
           'max_iter':Integer(50,500),'max_leaf_nodes':Integer(5,100),'max_depth':Integer(3,10),
            'max_bins':Integer(5,255),'l2_regularization':Continuous(0.0001,1000)}

xgb_reg = xgb.XGBRegressor()
cv=TimeSeriesSplit(n_splits=5)
hgbr=HistGradientBoostingRegressor()
xgb_ga=GASearchCV(estimator=xgb_reg,
                        cv=cv,
                        scoring='neg_mean_squared_error',
                        population_size=3,
                        generations=2,
                        tournament_size=2,
                        elitism=True,
                        crossover_probability=0.8,
                        mutation_probability=0.1,
                        param_grid=param_grid,
                        criteria='max',
                        algorithm='eaMuPlusLambda',
                        n_jobs=-1,
                        verbose=True)

xgb_ga.fit(X_train,y_train)
xgb_opt=xgb.XGBRegressor(xgb_ga.best_params_)
#hgbr_opt=HistGradientBoostingRegressor(**hgbr_ga.best_params_)
def train_predict(model,X_train,y_train,X_test,y_test,steps,sample_weight):
    model.fit(X_train,y_train,sample_weight)
    forecast=[]
    for k in range(len(y_test)):
        X_input=list(X_test[k,:4])
        for i in range(steps):
            if(k+i<steps):
                X_input.append(X_train[-1][k+i])
            else:
                X_input.append(forecast[-(steps-k-i)])
        forecast.append(round(max(model.predict(np.array(X_input).reshape((1,len(X_input))))[0],0)))
    return forecast

forecast=train_predict(xgb_opt,X_train,y_train,X_test,y_test,lags,sample_weight)

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	3     	-1673.84	962.934    	-312.132   	-2367.82   
1  	6     	-979.859	949.858    	-304.291   	-2323.15   
2  	4     	-312.132	0          	-312.132   	-312.132   
[01:13:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "n_estimadors" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBoostError: [01:13:16] C:\Users\Administrator\workspace\xgboost-win64_release_1.5.1\src\objective\objective.cc:26: Unknown objective function: `{'n_estimadors': 327, 'max_depth': 7, 'learning_rate': 0.7585850014840604, 'booster': 'gbtree', 'reg_lambda': 0.5756854050595877}`
Objective candidate: survival:aft
Objective candidate: binary:hinge
Objective candidate: multi:softmax
Objective candidate: multi:softprob
Objective candidate: rank:pairwise
Objective candidate: rank:ndcg
Objective candidate: rank:map
Objective candidate: reg:pseudohubererror
Objective candidate: binary:logistic
Objective candidate: binary:logitraw
Objective candidate: reg:linear
Objective candidate: count:poisson
Objective candidate: survival:cox
Objective candidate: reg:gamma
Objective candidate: reg:tweedie
Objective candidate: reg:squarederror
Objective candidate: reg:squaredlogerror
Objective candidate: reg:logistic


In [25]:

def get_index_aberto(X_test):
    fechado_array=X_test[:,3]
    index_list=[]
    
    for i in range(len(fechado_array)):
        if(fechado_array[i]==0):
            index_list.append(i)
    return index_list

In [26]:
index_aberto=get_index_aberto(X_test)
forecast=np.array(forecast)[index_aberto]
y_test=y_test[index_aberto]

In [27]:
print(r2_score(y_test,forecast), mean_squared_error(y_test,forecast))

0.9112506363279747 861.6536796536797


In [12]:
hgbr_opt.get_params

<bound method BaseEstimator.get_params of HistGradientBoostingRegressor(l2_regularization=126.67965348917133,
                              learning_rate=0.13273050446279072, max_bins=51,
                              max_depth=6, max_iter=367, max_leaf_nodes=32)>

In [35]:
X=forecast_data_final.iloc[:,:-1].values
y=forecast_data_final.iloc[:,-1].values
X_train=X[:-1440,:]
y_train=y[:-1440]
X_test=X[-1440:]
y_test=y[-1440:]

rfr=RandomForestRegressor(bootstrap=True, max_features=0.8, min_samples_leaf=1, min_samples_split=13, n_estimators=100,oob_score=True,n_jobs=-1)
def train_predict(model,X_train,y_train,X_test,steps,best_features,non_lag_features_index,best_features_lags):
    model.fit(X_train[:,best_features],y_train)
    forecast=[]
    for k in range(len(y_test)):
        X_input=list(X_test[k,best_features[non_lag_features_index]])
        for i in range(steps):
            if(k+i<steps):
                X_input.append(X_train[-1][k+i])
            else:
                X_input.append(forecast[-(steps-k-i)])
        X_input=np.array(X_input).reshape((1,len(X_input)))
        X_input=X_input[:,best_features]
        forecast.append(round(max(model.predict(X_input)[0],0)))
    return forecast

forecast=train_predict(rfr,X_train,y_train,X_test,lags,best_features,non_lag_features_index,best_features_lags)

NameError: name 'best_features' is not defined

In [233]:
fig = px.histogram(pd.DataFrame(dict(zip(forecast_data_final.columns[:-1],rfr.feature_importances_)).items(),columns=['features','importance']),x='features',y='importance')
fig.show()

In [95]:
len(forecast)

693

In [14]:
def get_index_aberto(X_test):
    fechado_array=X_test[:,3]
    index_list=[]
    
    for i in range(len(fechado_array)):
        if(fechado_array[i]==0):
            index_list.append(i)
    return index_list

np.array(forecast)[get_index_aberto(X_test)]

array([ 33,  33,  33,  32,  32, 126, 198, 263, 273, 261, 187,  98,  54,
        36,  37,  36,  35,  37,  83,  41,  42,  42,  41,  46, 161, 187,
       255, 254, 254, 186,  94,  84,  41,  42,  41,  41,  42, 161, 187,
       233, 274, 279, 179,  91,  84,  38,  42,  42,  41,  42, 150, 195,
       245, 254, 249, 193,  90,  79,  46,  50,  48,  47,  42, 138, 188,
       237, 244, 239, 161,  88,  69,  47,  50,  49,  47,  56, 104, 167,
       207, 228, 204, 115,  76,  52,  45,  44,  44,  44,  44, 116, 169,
       217, 205, 154, 107,  74,  50,  37,  45,  44,  44,  48,  77,  42,
        49,  49,  49,  64, 130, 182, 215, 215, 185, 145, 101,  68,  42,
        50,  50,  50,  54, 131, 179, 220, 211, 186, 144,  81,  63,  53,
        53,  52,  52,  56, 128, 185, 211, 191, 163, 137,  81,  63,  60,
        62,  60,  60,  51, 118, 174, 206, 192, 173, 117,  77,  53,  28,
        59,  59,  57,  57,  61, 107, 168, 203, 203, 162, 109,  70,  51,
        52,  47,  44,  45,  47, 107, 152, 176, 174, 122,  87,  5

In [29]:
forecast=np.array(forecast)[get_index_aberto(X_test)]

In [30]:
y_test=y_test[get_index_aberto(X_test)]

In [45]:
def train_predict(model,X_train,y_train,X_test,steps):
    model.fit(X_train,y_train)
    forecast=[]
    for k in range(len(y_test)):
        X_input=list(X_test[k,:3])
        for i in range(steps):
            if(k+i<steps):
                X_input.append(X_train[-1][k+i])
            else:
                X_input.append(forecast[-(steps-k-i)])
        forecast.append(round(max(model.predict(np.array(X_input).reshape((1,len(X_input))))[0],0)))
    return forecast

cv = TimeSeriesSplit(n_splits=5)
model = TPOTRegressor(generations=5, population_size=20, scoring='neg_mean_absolute_error', cv=cv, verbosity=2, random_state=1, n_jobs=-1)
forecast=train_predict(model,X_train,y_train,X_test,lags)

Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]



TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.


RuntimeError: A pipeline has not yet been optimized. Please call fit() first.

In [ ]:
model.export('modelo.py')

In [57]:
def sub(arr1,arr2,value):
    for i in range(len(arr1)):
        if arr1[i]==value:
            arr2[i]=value
    return

sub(y_test,forecast,0)

In [191]:
df_plot=pd.DataFrame.from_dict({'x':[i for i in range(len(forecast))],'Previsão':forecast,'Real':y_test})
df_plot.columns



Index(['x', 'Previsão', 'Real'], dtype='object')

In [192]:
import plotly

fig = px.line(df_plot[720:],x='x',y=df_plot.columns[1:],title='Previsões loja 37',                 labels={
                     "x": "Hora",
                     "value": "#tickets"
                 })
fig.show()

In [34]:
forecast

array([], dtype=int32)

In [32]:
index_aberto=get_index_aberto(X_test)
forecast=np.array(forecast)[index_aberto]
y_test=y_test[index_aberto]

In [33]:
print(r2_score(y_test,forecast), mean_squared_error(y_test,forecast))

ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [ ]:
def maape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.arctan(np.abs((y_true-y_pred)/y_true)))


In [ ]:
print(r2_score(Y_test,pred), mean_squared_error(Y_test,pred))

## prophet

In [11]:
forecast_data_prophet=forecast_data.rename(columns={'fechaHoraInicio':'ds','Mês':'add1','Dia':'add2','Hora':'add3','Fechado':'add4','tickets':'y'})
forecast_data_prophet.head(5)

,ds,y,add1,add2,add3,add4
0,2015-01-02 10:00:00,199.0,1,4,10,0
1,2015-01-02 11:00:00,268.0,1,4,11,0
2,2015-01-02 12:00:00,337.0,1,4,12,0
3,2015-01-02 13:00:00,390.0,1,4,13,0
4,2015-01-02 14:00:00,249.0,1,4,14,0


In [16]:
df.shape

(43784, 6)

In [1]:

from prophet import Prophet
m = Prophet(daily_seasonality=True)
m.add_regressor('add1')
m.add_regressor('add2')
m.add_regressor('add3')
m.add_regressor('add4')
df_train=forecast_data_prophet[:-1440]
df_test=forecast_data_prophet[-1440:]
m.fit(df_train)
future = m.make_future_dataframe(periods=1440,freq='H')

NameError: name 'forecast_data_prophet' is not defined

In [18]:
future = df_test.drop('y', 1)
df_test

,ds,y,add1,add2,add3,add4
42350,2019-11-02 00:00:00,0.0,11,5,0,1
42351,2019-11-02 01:00:00,0.0,11,5,1,1
42352,2019-11-02 02:00:00,0.0,11,5,2,1
42353,2019-11-02 03:00:00,0.0,11,5,3,1
42354,2019-11-02 04:00:00,0.0,11,5,4,1
...,...,...,...,...,...,...
43785,2019-12-31 19:00:00,146.0,12,1,19,0
43786,2019-12-31 20:00:00,0.0,12,1,20,1
43787,2019-12-31 21:00:00,0.0,12,1,21,1
43788,2019-12-31 22:00:00,0.0,12,1,22,1


In [19]:
forecast=m.predict(future)['yhat'].values

def get_index_aberto_prophet(df_test):
    fechado_array=df_test.values[:,-1]
    index_list=[]
    
    for i in range(len(fechado_array)):
        if(fechado_array[i]==0):
            index_list.append(i)
    return index_list

In [20]:
forecast=forecast[get_index_aberto_prophet(df_test)]
y_test=df_test['y'].values
y_test=y_test[get_index_aberto_prophet(df_test)]

In [21]:
print(len(y_test))
print(len(forecast))


693
693


In [22]:
print(r2_score(y_test,forecast), mean_squared_error(y_test,forecast))

0.8174635766633012 1772.2175611323908


In [55]:
df_plot=pd.DataFrame.from_dict({'x':[i for i in range(len(forecast))],'Previsão':forecast,'Real':y_test})
fig = px.line(df_plot[720:],x='x',y=df_plot.columns[1:],title='Previsões loja 37',                 labels={
                     "x": "Hora",
                     "value": "#tickets"
                 })
fig.show()

### Stacking

In [48]:
import json
con,cursor=connect_sqlserver()

r=cursor.execute("""SELECT top 5 model_metrics
  FROM [sisqualFORECASTDATA].[Martim].[models]
  where tienda=37
order by best_metric desc""")

lst_stacking=[]
for row in r:
    d=json.loads(row.model_metrics)
    y_pred=json.loads(d['y_pred'])
    lst_stacking.append(y_pred)
    y_true=np.array(json.loads(d['y_true']))

In [49]:
np_stacking=np.array(lst_stacking).T
np_stacking.shape

(693, 5)

In [121]:
from sklearn.linear_model import Ridge
from sklearn.svm import LinearSVR
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import RidgeCV
N=round(np_stacking.shape[0]*0.7)
np_stacking_train=np_stacking[:N,]
y_true_train=y_true[:N]
np_stacking_test=np_stacking[N:,]
y_true_test=y_true[N:]
clf = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1,10,100,1000,10000],cv=cv)
#clf=LinearSVR()
clf.fit(np_stacking_train, y_true_train)
#cv = TimeSeriesSplit(n_splits=10)
#tpot = TPOTRegressor(generations=50, population_size=1000, scoring='neg_mean_absolute_error', cv=cv, verbosity=2, random_state=1, n_jobs=-1)
#tpot.fit(np_stacking_train,y_true_train)

RidgeCV(alphas=array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04]),
        cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=None))

In [34]:
tpot.export('modelostacking.py')

In [122]:
clf.get_params()

{'alpha_per_target': False,
 'alphas': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04]),
 'cv': TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=None),
 'fit_intercept': True,
 'gcv_mode': None,
 'normalize': 'deprecated',
 'scoring': None,
 'store_cv_values': False}

In [116]:
json.dumps(clf.get_params())

TypeError: Object of type ndarray is not JSON serializable

In [114]:
str(clf.get_params())

"{'C': 1.0, 'dual': True, 'epsilon': 0.0, 'fit_intercept': True, 'intercept_scaling': 1.0, 'loss': 'epsilon_insensitive', 'max_iter': 1000, 'random_state': None, 'tol': 0.0001, 'verbose': 0}"

In [89]:
y_pred_stacking=clf.predict(np_stacking_test)
print(r2_score(y_true_test,y_pred_stacking), mean_squared_error(y_true_test,y_pred_stacking))

0.9073722727784503 958.9021326857523


In [52]:
[i for i in range(len(y_true_test))]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [126]:
np.round(y_pred_stacking)

array([278., 343., 350., 242., 113.,  89., 111., 143., 165., 148.,  48.,
       112., 201., 283., 342., 365., 242., 114.,  64.,  78., 101., 117.,
       110.,  42.,  66., 141., 198., 216., 202.,  68.,  10., 105., 175.,
       241., 293., 295., 156.,  73.,  80., 111., 144., 177., 149.,  44.,
       144., 209., 293., 352., 378., 244., 104.,  92., 126., 176., 193.,
       156.,  44., 148., 214., 288., 344., 366., 236., 105.,  90., 121.,
       166., 178., 146.,  44., 143., 207., 286., 345., 372., 241., 107.,
        89., 121., 159., 183., 157.,  47., 131., 197., 278., 343., 350.,
       242., 113.,  89., 111., 143., 165., 148.,  48., 112., 201., 283.,
       342., 365., 242., 114.,  64.,  78., 101., 117., 110.,  42.,  66.,
       141., 198., 216., 202.,  68., 160., 201., 275., 342., 365., 233.,
       103.,  96., 126., 175., 200., 162.,  47., 144., 209., 293., 352.,
       378., 244., 104.,  92., 126., 176., 193., 143., 207., 286., 345.,
       372., 241., 107.,  89., 121., 159., 183., 15